In [2]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
!pip install -U awswrangler pyqt5 pyqtwebengine bokeh ipython pylint
!pip install sagemaker-data-insights==0.4.0
import awswrangler as wr

  Using cached awswrangler-3.5.2-py3-none-any.whl.metadata (23 kB)
  Using cached PyQt5-5.15.10-cp37-abi3-manylinux_2_17_x86_64.whl.metadata (2.1 kB)
  Using cached PyQtWebEngine-5.15.6-cp37-abi3-manylinux1_x86_64.whl (230 kB)
  Using cached bokeh-3.3.4-py3-none-any.whl.metadata (12 kB)
  Using cached ipython-8.21.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached pylint-3.0.3-py3-none-any.whl.metadata (12 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached PyQt5_sip-12.13.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (504 bytes)
  Using cached PyQt5_Qt5-5.15.2-py3-none-manylinux2014_x86_64.whl (59.9 MB)
  Using cached PyQtWebEngine_Qt5-5.15.2-py3-none-manylinux2014_x86_64.whl (67.5 MB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached awswrangler-3.5.2-py3-none-any.whl (402 kB)
Using cached PyQt5-5.15.10-cp37-abi3-manylinux_2_17_x86_64.whl (8.2 MB)
Using cached bokeh-3.3.4-py3-none-any.whl (6.8 MB

In [6]:
import pandas as pd

In [8]:
def train_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 4)

def val_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 5) and (int(partitions["month"]) > 4)


In [10]:
path = "s3://{}/meterdataset.parquet/".format(bucket)
df_parquet_train = wr.s3.read_parquet(
    path, 
    #columns=["artists", "track_name", "popularity"], 
    partition_filter=train_filter, 
    dataset=True
)
df_parquet_train.shape

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


(2904, 7)

In [11]:
path = "s3://{}/meterdataset.parquet/".format(bucket)
df_parquet_val = wr.s3.read_parquet(
    path, 
    #columns=["artists", "track_name", "popularity"], 
    partition_filter=val_filter, 
    dataset=True
)
df_parquet_val.shape

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


(744, 7)

In [14]:
df_parquet_train = df_parquet_train.drop(columns=['year','month'])
df_parquet_val = df_parquet_val.drop(columns=['year','month'])
df_parquet_train

,kWh,weekday,hour,temperature,Holiday
timestamp,,,,,
2012-01-01 00:00:00,0.230723,6,0.0,12.12,-1.0
2012-01-01 01:00:00,0.268496,6,1.0,12.59,-1.0
2012-01-01 02:00:00,0.228618,6,2.0,12.45,-1.0
2012-01-01 03:00:00,0.191603,6,3.0,12.03,-1.0
2012-01-01 04:00:00,0.144564,6,4.0,12.04,-1.0
...,...,...,...,...,...
2012-04-30 19:00:00,0.275060,0,19.0,14.27,-1.0
2012-04-30 20:00:00,0.301409,0,20.0,13.44,-1.0
2012-04-30 21:00:00,0.286507,0,21.0,12.55,-1.0


In [15]:
train_string = "housepow_train.parquet"
val_string = "housepow_val.parquet"
df_parquet_train.to_parquet(train_string)
df_parquet_val.to_parquet(val_string)

In [13]:
prefix = "sagemaker/DEMO-xgboost-parquet"
use_amt = True

In [17]:
%%time
sess.upload_data(
    train_string, bucket=bucket, key_prefix=prefix + "/" + "training"
)

sess.upload_data(
    val_string, bucket=bucket, key_prefix=prefix + "/" + "validation"
)

CPU times: user 24.9 ms, sys: 4.61 ms, total: 29.5 ms
Wall time: 277 ms


's3://sagemaker-us-east-1-645257835735/sagemaker/DEMO-xgboost-parquet/validation/housepow_val.parquet'

In [18]:
training_image = sagemaker.image_uris.retrieve("xgboost", region, "1.0-1")

#### Construct a SageMaker generic estimator using the SageMaker XGBoost container

In [36]:
dataset_uri_prefix = "s3://" + bucket + "/" + prefix + "/training/"
dataset_uri_prefix_val = "s3://" + bucket + "/" + prefix + "/validation/"
training_output_path = "s3://" + bucket + "/" + prefix + "/training_output"

In [37]:
from sagemaker.estimator import Estimator

training_model = Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=5,
    max_run=3600,
    input_mode="File",
    output_path=training_output_path,
    sagemaker_session=sess,
)

#### Set hyperparameters

In [38]:
training_model.set_hyperparameters(objective="reg:linear", num_round=50)

#### Specify training dataset 
Specify the training dataset created in the [Build Training Dataset](#Build-Training-Dataset) section.

In [39]:
train_data = sagemaker.inputs.TrainingInput(
    dataset_uri_prefix,
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

validation_data = sagemaker.inputs.TrainingInput(
    dataset_uri_prefix_val,
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, 'validation': validation_data}

#### Start training

In [40]:
training_model.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-02-02-03-34-21-238


2024-02-02 03:34:21 Starting - Starting the training job...
2024-02-02 03:34:36 Starting - Preparing the instances for training.........
2024-02-02 03:35:59 Downloading - Downloading input data...
2024-02-02 03:36:34 Downloading - Downloading the training image...
2024-02-02 03:37:15 Training - Training image download completed. Training in progress.
2024-02-02 03:37:15 Uploading - Uploading generated training model[2024-02-02 03:37:06.666 ip-10-0-227-93.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Single node training.
[2024-02-02 03:37:06.722 ip-10-0-227-93.ec2.internal:7 INFO json_config.py:91] 

In [46]:
#predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2024-01-29-06-33-12-725
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-01-29-06-33-12-725
